In [1]:
import sys
sys.path.insert(0, "..")

import jax
import jax.numpy as jnp
import jax.random
import flax.linen as nn
import flax
from jax import grad, jit, vmap
from matplotlib import pyplot as plt


import module.samplers
import module.hamiltonians
import module.wavefunctions

In [2]:
key = jax.random.PRNGKey(0)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
def potential(x):
    return 0.5 * jnp.sum(x**2, axis = -1)*0

In [5]:
class NN(nn.Module):

    @nn.compact
    def __call__(self, x):
        
        # A tensor of variational parameters is defined by calling
        # the method `self.param` where the arguments will be:
        # - arbitrary name used to refer to this set of parameters
        # - an initializer used to provide the initial values. 
        # - The shape of the tensor
        # - The dtype of the tensor.
        a = self.param(
            "a", nn.initializers.normal(), (), float   # scalar
        )
        
        # compute the log amplitude
        # here we use exp[-r/a]
        logpsi = -jnp.sqrt(jnp.sum(x**2, axis = -1)) / jnp.abs(a)

        # sum the output
        return -jnp.sum(x**2, axis = -1)

In [6]:
class SimpleOrbital(module.wavefunctions.Wavefunction):
    def __init__(self):
        self.model = NN()
        self.parameters = self.model.init(key, jnp.empty(3,))

    def calc_logpsi(self, x):
        return self.model.apply(self.parameters, x)

In [10]:
orbital = SimpleOrbital()
sampler = module.samplers.MCMC(orbital.calc_logprob, shape = (3,), variance = 1)
H = module.hamiltonians.Particles(masses = [1], potential = potential)

In [35]:
samples = sampler.sample(key, jnp.array([0.,0.,0.,]), 1500)

In [36]:
E_loc = H.calc_H_loc(orbital, samples)

In [37]:
jnp.mean(E_loc)

Array(1.0611291, dtype=float32)